# Два типа сравнения контекстов

In [1]:
import openpyxl
import re
import pymystem3
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
import gensim
import logging
import nltk.data
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec
import warnings
warnings.filterwarnings('ignore')
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords

## Работа с таблицей по векам

In [2]:
wb = openpyxl.load_workbook('/Users/annaaksenova/Desktop/Работа к Кувшинской/То есть выборка по векам.xlsx')
sheet = wb['21'] # Имя листа

In [3]:
m = Mystem()

In [4]:
v1 = []
# Собираем примеры из таблицы
for v in sheet.iter_rows(min_row=2, min_col=21, max_col=21, max_row=1001, values_only=True):
    if v[0] == None:
        v1.append('')
    else:
        v1.append(v[0].replace(u'\xa0', ' '))
v2 = []
for v in sheet.iter_rows(min_row=2, min_col=22, max_col=22, max_row=1001, values_only=True):
    if v[0] == None:
        v2.append('')
    else:
        item = v[0].replace(u'\xa0', ' ')
        item = re.sub('\[.+?\]', '', item)
        item = re.sub('[Тт]о есть', 'тоесть', item) # \b убирает случаи что есть
        v2.append(item)
values = [v1[i]+v2[i] for i in range(1000)]
print(values[:10])

[' Я засунула пластинку и блюдо в один пакет, положила под сиденье и ― надо знать наши самолёты советских времён, лишённые кондиционеров, ― в конце пути в Ереван достала из-под сиденья пластинку, принявшую форму таза, тоесть блюда.   ', ' Само название места происходит от того, что здесь врачевали «косимых» ― тоесть увечных.   ', ' Я даже не могу найти концов, тоесть понять, кто же мне, собственно, должен платить.   ', 'Никаких провокаций ― известные имена-брэнды.  Расчёт на то, что, даже если зритель не шибко приобщён к балету и не знает, кто такая Аньес Летестю, он непременно отреагирует на Opera National de Paris.  Или хотя бы на «Лебединое озеро». Или хотя бы на Paris.  Плюс гарантия качества: звёзды подобраны заслуженные, но при этом, что важно, действующие: спешно доставленные на пароходе прямо из Парижа, поднимешь крышку ― пар! тоесть готовые доказывать свой статус каждым спектаклем.   ', '  «Чтобы не быть выдавленными с рынка, ― говорит академик Алексей Макаров, ― необходимы же

In [5]:
tagged_corp = []
for sentence in values:
    ana = m.analyze(sentence)
    tagged_corp.append(ana)

In [12]:
final = []
count = 1
for sent in tagged_corp:
    sentence = []
    for word in sent:
        if 'analysis' in word and word['analysis'] != []:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            lex = word['analysis'][0]['lex']
            token = word['text']
            if pos == 'V' or pos == 'S' or pos == 'A' or pos == 'ADV':
                sentence.append((token, lex, pos))
    if sentence == []:
        print(sent)
        print(count)
    else:
        final.append(sentence)
    count+=1

In [8]:
final[:10]

[[('засунула', 'засовывать', 'V'),
  ('пластинку', 'пластинка', 'S'),
  ('блюдо', 'блюдо', 'S'),
  ('пакет', 'пакет', 'S'),
  ('положила', 'полагать', 'V'),
  ('сиденье', 'сидение', 'S'),
  ('надо', 'надо', 'ADV'),
  ('знать', 'знать', 'V'),
  ('самолёты', 'самолет', 'S'),
  ('советских', 'советский', 'A'),
  ('времён', 'время', 'S'),
  ('лишённые', 'лишать', 'V'),
  ('кондиционеров', 'кондиционер', 'S'),
  ('конце', 'конец', 'S'),
  ('пути', 'путь', 'S'),
  ('Ереван', 'ереван', 'S'),
  ('достала', 'доставать', 'V'),
  ('сиденья', 'сидение', 'S'),
  ('пластинку', 'пластинка', 'S'),
  ('принявшую', 'принимать', 'V'),
  ('форму', 'форма', 'S'),
  ('таза', 'таз', 'S'),
  ('тоесть', 'тоедать', 'V'),
  ('блюда', 'блюдо', 'S')]]

In [13]:
len(final)

1000

In [14]:
with open('sentences.txt', 'w') as fw:
    for sentence in final:
        for word in sentence:
            for part in word:
                fw.write(part + ' ')
        fw.write('\n')

## Сравнение средних векторов правого и левого контекстов

Здесь я брала именно не сумму, а среднее, потому что количество слов разное

In [15]:
import re
part_sent = []

with open('part_sim.txt') as fh: 
    lines = fh.readlines()
for line in lines:
    line = line.strip()
    number, cos = line.split()
    part_sent.append([number, cos])


In [16]:
wb = openpyxl.load_workbook('/Users/annaaksenova/Desktop/Работа к Кувшинской/То есть выборка по векам.xlsx')
sheet = wb['21']
for i in part_sent: # для каждой пары номер-косинус вытаскиваем из таблицы разметку
    i.append(sheet['Q{}'.format(str(int(i[0])+1))].value)
    i.append(sheet['P{}'.format(str(int(i[0])+1))].value)

In [17]:
from collections import Counter

Сколько примеров каждого типа имеют схожесть контекста > 0.6

In [18]:
tags_near = [i[2] for i in part_sent if float(i[1]) > 0.6]
tags_near_common = Counter(tags_near)
print(len(tags_near))
tags_near_common

145


Counter({'определение': 12,
         'следствие': 51,
         'дополнение': 13,
         'обобщение': 6,
         'спецификация': 9,
         'переспрос инт': 7,
         'интерпретация': 22,
         'переформулирование': 18,
         'коррекция': 4,
         'пример': 2,
         'переспрос как': 1})

Сколько примеров каждого типа имеют схожесть контекста < 0.4

In [19]:
tags_far = [i[2] for i in part_sent if float(i[1]) < 0.4]
tags_far_common = Counter(tags_far)
print(len(tags_far))
tags_far_common

520


Counter({'переформулирование': 64,
         'интерпретация': 136,
         'следствие': 113,
         'дополнение': 43,
         'обобщение': 22,
         'определение': 35,
         'переспрос инт': 27,
         'спецификация': 33,
         'переспрос': 7,
         'коррекция': 19,
         'обр определение': 3,
         'дискурс': 9,
         'модальность': 1,
         'пример': 4,
         'переспрос как': 4})

In [20]:
part_sent[:10]

[['292', '0.8937959', 'определение', 1],
 ['215', '0.8420358', 'следствие', 2],
 ['301', '0.83674455', 'дополнение', 2],
 ['557', '0.8298659', 'следствие', 2],
 ['953', '0.81978774', 'следствие', 2],
 ['62', '0.81574357', 'обобщение', 2],
 ['505', '0.8127927', 'спецификация', 2],
 ['838', '0.8119556', 'следствие', 2],
 ['365', '0.8048143', 'следствие', 2],
 ['598', '0.80335826', 'определение', 1]]

Считаем медианы близости для каждого типа значений

In [21]:
import numpy as np

In [22]:
medians = {}
for x in part_sent:
    if x[2] not in medians.keys():
        medians[x[2]] = []
    medians[x[2]].append(float(x[1]))
for meaning, sim in medians.items():
    medians[meaning] = np.median(sim)

In [23]:
medians

{'определение': 0.432113,
 'следствие': 0.43812629000000003,
 'дополнение': 0.39248097,
 'обобщение': 0.41474432,
 'спецификация': 0.35622537,
 'переспрос инт': 0.38275018,
 'интерпретация': 0.37294193,
 'переформулирование': 0.36548322,
 'коррекция': 0.40758401499999997,
 'пример': 0.373233,
 'переспрос как': 0.29097784,
 'дискурс': 0.17193993,
 'модальность': 0.41158122,
 'обр определение': 0.3304534,
 'переспрос': 0.21245712}

## Медианы попарной близости

Здесь я проделала то же самое, только выбрала самые похожие слова. Я работала с векторами на сервере, а потом выкачала только близость

In [26]:
with open('word_sim.txt') as fh:
    cos = [[float(i.strip())] for i in fh.readlines()]

In [27]:
wb = openpyxl.load_workbook('/Users/annaaksenova/Desktop/Работа к Кувшинской/То есть выборка по векам.xlsx')
sheet = wb['21']
for i in range(len(cos)): # для каждой пары номер-косинус берем из таблицы разметку
    cos[i].append(sheet['Q{}'.format(str(i+2))].value)

In [28]:
medians_words = {}
for x in cos:
    if x[1] not in medians_words.keys():
        medians_words[x[1]] = []
    medians_words[x[1]].append(float(x[0]))
for meaning, sim in medians_words.items():
    medians_words[meaning] = np.median(sim)

In [29]:
medians_words

{'коррекция': 1.0,
 'определение': 0.5928247,
 'интерпретация': 0.5599262300000001,
 'следствие': 0.76763475,
 'дополнение': 0.6419779,
 'переспрос как': 0.28707352,
 'переформулирование': 0.5536154,
 'переспрос инт': 0.64760023,
 'спецификация': 0.55591154,
 'пример': 0.487336125,
 'модальность': 1.0000001,
 'обобщение': 0.61480975,
 'дискурс': 0.327046635,
 'переспрос': 0.33812416,
 'обр определение': 0.3905912}

In [30]:
sorter = [(key, value) for key,value in medians_words.items()]

In [31]:
sorted(sorter, reverse=True, key = lambda x: x[1])

[('модальность', 1.0000001),
 ('коррекция', 1.0),
 ('следствие', 0.76763475),
 ('переспрос инт', 0.64760023),
 ('дополнение', 0.6419779),
 ('обобщение', 0.61480975),
 ('определение', 0.5928247),
 ('интерпретация', 0.5599262300000001),
 ('спецификация', 0.55591154),
 ('переформулирование', 0.5536154),
 ('пример', 0.487336125),
 ('обр определение', 0.3905912),
 ('переспрос', 0.33812416),
 ('дискурс', 0.327046635),
 ('переспрос как', 0.28707352)]